In [8]:
import praw
import pandas as pd
from datetime import datetime

SECRET = "fDTjgQd2sS6gmVrqm7IcPPhp05NHXg"
APP_ID = "KGv2MMUcdlqidiWTIP4YPA"

reddit = praw.Reddit(
        client_id=APP_ID,
        client_secret=SECRET,
        user_agent="Comment Extraction")

search_query = "Margot Robbie"
max_comments = 10000

comment_bodies = []
comment_authors = []
comment_upvotes = []
comment_awards = []
comment_dates = []

# Define the date range from 21st July to 27th July
start_date = datetime(2023, 7, 21)
end_date = datetime(2023, 7, 27)

for submission in reddit.subreddit("all").search(search_query, limit=max_comments):
    submission.comments.replace_more(limit=0)  # Get only top-level comments
    for comment in submission.comments:
        if comment.is_root:
            # Convert the comment date from UTC timestamp to datetime object
            comment_date = datetime.utcfromtimestamp(comment.created_utc)
            # Check if the comment falls within the specified date range
            if start_date <= comment_date <= end_date:
                comment_bodies.append(comment.body)
                comment_authors.append(comment.author.name if comment.author else "[deleted]")
                comment_upvotes.append(comment.score)
                comment_awards.append(comment.total_awards_received)
                comment_dates.append(comment_date)

data = {
    "Comment_Body": comment_bodies,
    "Author": comment_authors,
    "Upvotes": comment_upvotes,
    "Awards": comment_awards,
    "Date": comment_dates,
}

df = pd.DataFrame(data)

df = df[df['Author'] != 'AutoModerator']
df = df[~df['Comment_Body'].str.contains(r'\[Banned for overrating\]', case=False)]
df = df[~df['Comment_Body'].str.contains(r'\br/\w+', case=False)]

df

,Comment_Body,Author,Upvotes,Awards,Date
0,"I had to mute the video to enjoy it. Like, shu...",Ok_Shirt_3481,7392,0,2023-07-25 03:42:18
1,"Margot, margot, MaRgOt, mArGoT, MARGOT!!!",mr-PicklePants,4859,1,2023-07-25 03:17:56
2,I was so grateful to learn sign language as pa...,Queenside22,3782,0,2023-07-25 04:03:15
3,“Margot! Margot!” …. she’s having a conversati...,hemansteve,13596,3,2023-07-25 03:20:43
5,She's clearly using an Australian accent when ...,RepresentativeNice22,4123,0,2023-07-25 03:01:34
...,...,...,...,...,...
1644,People need to stop because there is NO way.,nghtmareb4coffee,4,0,2023-07-24 23:59:03
1645,I’ve gotten this comment and I look nothing li...,FSU2016,1,0,2023-07-25 01:20:09
1646,"I don't know what's hotter, her perfect tongue...",GoonedAndEdged,9,0,2023-07-26 15:08:41
1647,She loves showing it off,FelixTanim,5,0,2023-07-26 15:40:24


In [9]:
# Specify the file path where you want to save the CSV file
csv_file_path = "reddit_comments_Margot.csv"

# Save the DataFrame to a CSV file
df.to_csv(csv_file_path, index=False)